# Tutorial of LangDa

### Schema of Facts and Views

##### 1.1 Define FactSchema

In [1]:
# === Define FactSchema ===
# Usage:
# - FactSchema is the canonical set of fact-layer predicates allowed in your system.
# - Each predicate is defined with a signature: a list of ArgSpec entries.
#
# Design rationale:
# - We separate names and argument signatures so that predicates are stable, hashable, and
#   unambiguous even if names collide across domains (namespace/role).
# - This enables FactView filtering and stable predicate_id references for LLM decoding.
#
# ArgSpec fields:
# - datatype: the data type of the argument, e.g., string/int/float
# - namespace: business domain/module/ontology (optional)
# - role: semantic role of the parameter (optional)
import pprint as pp
from new_symbolic_agent.ir.fact_schema import ArgSpec, PredicateSchema, FactSchema

person_name = ArgSpec("string", namespace="person", role="name")
city_name = ArgSpec("string", namespace="geo", role="name")
company_name = ArgSpec("string", namespace="org", role="name")
country_name = ArgSpec("string", namespace="geo", role="name")

person = PredicateSchema("person", 1, [person_name], description="A person entity")
city = PredicateSchema("city", 1, [city_name], description="A city entity")
company = PredicateSchema("company", 1, [company_name], description="A company entity")
country = PredicateSchema("country", 1, [country_name], description="A country entity")

lives = PredicateSchema(
    "lives_in", 2, [person_name, city_name], description="Relationship: person lives in city"
)
works = PredicateSchema(
    "works_at", 2, [person_name, company_name], description="Relationship: person works at company"
)
located = PredicateSchema(
    "located_in", 2, [city_name, country_name], description="Relationship: city located in country"
)
hq = PredicateSchema(
    "company_hq", 2, [company_name, city_name], description="Relationship: company HQ city"
)
friend = PredicateSchema(
    "are_friends", 2, [person_name, person_name], description="Relationship: person knows person"
)

schema = FactSchema([person, city, company, country, lives, works, located, hq, friend])
print("SCHEMA:")
pp.pprint(schema.to_dict())


SCHEMA:
{'predicates': [{'arity': 1,
                 'description': 'A person entity',
                 'name': 'person',
                 'schema_id': 'bbf463f16fe1f70b5f62b20fba41c9ee07615be5d35f61c792727314d3b9883b',
                 'signature': [{'datatype': 'string',
                                'namespace': 'person',
                                'role': 'name'}]},
                {'arity': 1,
                 'description': 'A city entity',
                 'name': 'city',
                 'schema_id': '95efd195cc4dafce381a135cc8177f2e4751818c77b8a94be7de2e9cdfea6c64',
                 'signature': [{'datatype': 'string',
                                'namespace': 'geo',
                                'role': 'name'}]},
                {'arity': 1,
                 'description': 'A company entity',
                 'name': 'company',
                 'schema_id': '8df463f324d38a63d5c0176ab99c543e6dba4ba555fba0c4c864f9578400a45d',
                 'signature': [{'datat

##### 1.2 Build Views

In [ ]:
# === Build FactView ===
# Usage:
# - FactView is the subset of FactSchema that a rule generator may reference.
# - Use it to constrain LLM outputs or rule construction to known predicates.
#
# Design rationale:
# - This isolates the rule-authoring surface from the full schema (security + clarity).
# - It also makes JSON schema constraints smaller and more stable.

import pprint as pp
from new_symbolic_agent.ir.filters import filter_from_dict

load_with_filters = False  # set to True to test Option B below
if not load_with_filters:
    # Option A: directly select schema_ids
    # view = schema.view([person.schema_id, lives.schema_id])
    view = schema.view([pred.schema_id for pred in schema.predicates()])

else:
    # Option B: Filter AST / dict sugar
    # - Filter AST supports combinators (And/Or/Not) and predicate matching.
    # - Dict sugar is a short-hand that compiles into the filter AST.
    filt = filter_from_dict({
    "and": [
        {"predicate": {"name": "person"}},
        {"predicate": {"name": "lives_in"}},
    ]
    })
    view = schema.view_from_filter(filt)

print("\nVIEW PREDICATES:")
pp.pprint(view.predicates())



VIEW PREDICATES:
[PredicateSchema(name='works_at',
                 arity=2,
                 signature=[ArgSpec(datatype='string',
                                    role='name',
                                    namespace='person'),
                            ArgSpec(datatype='string',
                                    role='name',
                                    namespace='org')],
                 description='Relationship: person works at company'),
 PredicateSchema(name='located_in',
                 arity=2,
                 signature=[ArgSpec(datatype='string',
                                    role='name',
                                    namespace='geo'),
                            ArgSpec(datatype='string',
                                    role='name',
                                    namespace='geo')],
                 description='Relationship: city located in country'),
 PredicateSchema(name='country',
                 arity=1,
                 signa

##### 1.3 Use of Filters on view

In [ ]:
import json
# === 2.1) Filter dict usage examples (all supported forms) ===
# Usage:
# - These dicts are parsed by filter_from_dict into an AST (And/Or/Not/PredMatch).
# - Use them with schema.view_from_filter(...) to build a FactView.
#
# Design rationale:
# - Dict sugar keeps user input minimal, while AST nodes enforce structure.

filter_examples = [
    {"name": "lives_in"},
    {"arity": 2},
    {"name": "lives_in", "arity": 2, "datatype": "string"},
    {"role": "subject"},
    {"namespace": "geo"},

    {"match": {"name": "lives_in"}},
    {"match": {"name": "lives_in", "arity": 2}},
    {"match": {"datatype": "string", "namespace": "geo"}},
    
    {"and": [{"match": {"name": "lives_in"}}, {"match": {"arity": 2}}]},
    {"or": [{"name": "person"}, {"name": "city"}]},
    {"not": {"match": {"namespace": "geo"}}},
    {
        "and": [
            {"match": {"arity": 2}},
            {"not": {"match": {"name": "lives_in"}}},
            {"or": [{"match": {"namespace": "geo"}}, {"match": {"namespace": "org"}}]},
        ]
    },
]

print("\nFILTER EXAMPLES:")
for idx, example in enumerate(filter_examples, start=1):
    filt = filter_from_dict(example)
    view_from_filter = schema.view_from_filter(filt)
    print(f"\nFilter {idx}: {json.dumps(example, ensure_ascii=False)}")
    print([pred.name for pred in view_from_filter.predicates()])


FILTER EXAMPLES:

Filter 1: {"name": "lives_in"}
['lives_in']

Filter 2: {"arity": 2}
['works_at', 'located_in', 'company_hq', 'lives_in', 'are_friends']

Filter 3: {"name": "lives_in", "arity": 2, "datatype": "string"}
['lives_in']

Filter 4: {"role": "subject"}
[]

Filter 5: {"namespace": "geo"}
['city', 'located_in', 'country']

Filter 6: {"match": {"name": "lives_in"}}
['lives_in']

Filter 7: {"match": {"name": "lives_in", "arity": 2}}
['lives_in']

Filter 8: {"match": {"datatype": "string", "namespace": "geo"}}
['city', 'located_in', 'country']

Filter 9: {"and": [{"match": {"name": "lives_in"}}, {"match": {"arity": 2}}]}
['lives_in']

Filter 10: {"or": [{"name": "person"}, {"name": "city"}]}
['city', 'person']

Filter 11: {"not": {"match": {"namespace": "geo"}}}
['works_at', 'company_hq', 'company', 'person', 'lives_in', 'are_friends']

Filter 12: {"and": [{"match": {"arity": 2}}, {"not": {"match": {"name": "lives_in"}}}, {"or": [{"match": {"namespace": "geo"}}, {"match": {"name

### Define Fact Instances

##### 2.1 Define facts directly

In [ ]:
# === 3) Manual facts (FactInstance) ===
# Usage:
# - FactInstance is a single fact record with a predicate_id and constant terms.
# - You can inject facts from any source, not just CSV.
#
# Design rationale:
# - Data access is abstracted by DataProvider, but manual facts are still a first-class path.
# - This is useful for programmatic data or unit tests.
import pprint as pp

from new_symbolic_agent.ir.expr_ir import Const
from new_symbolic_agent.fact_store.provider import FactInstance
from new_symbolic_agent.probability import ProbabilityConfig, resolve_probability

facts_from_user = [
    FactInstance(predicate_id=person.schema_id, terms=[Const("alice", "string")], prob=0.9),
    FactInstance(predicate_id=person.schema_id, terms=[Const("bob", "string")]),
    FactInstance(predicate_id=city.schema_id, terms=[Const("seattle", "string")]),
    FactInstance(predicate_id=company.schema_id, terms=[Const("openai", "string")]),
    FactInstance(predicate_id=country.schema_id, terms=[Const("usa", "string")]),
]

# Apply default probability policy (missing prob -> default)
# - ProbabilityConfig defines default value and missing policy.
# - "inject_default" will replace None with default_fact_prob.
prob_cfg = ProbabilityConfig(default_fact_prob=1.0, missing_prob_policy="inject_default")
facts_from_user = [
    FactInstance(
        predicate_id=f.predicate_id,
        terms=f.terms,
        prob=resolve_probability(
            f.prob,
            default_value=prob_cfg.default_fact_prob,
            policy=prob_cfg.missing_prob_policy,
            context="manual fact",
        ),
    )
    for f in facts_from_user
]
print("\nFACTS (manual):")
pp.pprint(facts_from_user)



FACTS (manual):
[FactInstance(predicate_id='bbf463f16fe1f70b5f62b20fba41c9ee07615be5d35f61c792727314d3b9883b',
              terms=[Const(value='alice', datatype='string')],
              prob=0.9),
 FactInstance(predicate_id='bbf463f16fe1f70b5f62b20fba41c9ee07615be5d35f61c792727314d3b9883b',
              terms=[Const(value='bob', datatype='string')],
              prob=1.0),
 FactInstance(predicate_id='95efd195cc4dafce381a135cc8177f2e4751818c77b8a94be7de2e9cdfea6c64',
              terms=[Const(value='seattle', datatype='string')],
              prob=1.0),
 FactInstance(predicate_id='8df463f324d38a63d5c0176ab99c543e6dba4ba555fba0c4c864f9578400a45d',
              terms=[Const(value='openai', datatype='string')],
              prob=1.0),
 FactInstance(predicate_id='a638a19b74d24af03cfe4694fccfe783c5a1fea967918a99668cd3c14004cbd1',
              terms=[Const(value='usa', datatype='string')],
              prob=1.0)]


##### 2.2 Load facts from csv files

In [ ]:
# === 4) CSVProvider facts ===
# Usage:
# - CSVProvider is a DataProvider implementation that loads facts from CSV.
# - Mapping from CSV columns to predicate arguments is explicit (CSVSource).
#
# Design rationale:
# - DataProvider abstracts access so we can add DBProvider later with the same API.
# - FactView filtering is handled by the provider, not hard-coded into CSV parsing.
from pathlib import Path

from new_symbolic_agent.fact_store.provider import CSVProvider, CSVSource
from new_symbolic_agent.probability import ProbabilityConfig

provider = CSVProvider(
    schema=schema,
    base_path=Path("new_problog_agent_parser/samples"),
    sources=[
        CSVSource(predicate_id=person.schema_id, file="people_rich.csv", columns=["name"]),
        CSVSource(predicate_id=city.schema_id, file="cities_rich.csv", columns=["name"]),
        CSVSource(predicate_id=company.schema_id, file="companies.csv", columns=["name"]),
        CSVSource(predicate_id=country.schema_id, file="countries.csv", columns=["name"]),
        CSVSource(predicate_id=lives.schema_id, file="lives_in_rich.csv", columns=["person", "city"]),
        CSVSource(predicate_id=works.schema_id, file="works_at.csv", columns=["person", "company"]),
        CSVSource(predicate_id=located.schema_id, file="located_in.csv", columns=["city", "country"]),
        CSVSource(predicate_id=hq.schema_id, file="company_hq.csv", columns=["company", "city"]),
        CSVSource(predicate_id=friend.schema_id, file="friends.csv", columns=["person_a", "person_b"]),
    ],
    prob_config=ProbabilityConfig(default_fact_prob=1.0, missing_prob_policy="inject_default"),
)
facts_from_csv = provider.query(view)
print("\nFACTS (csv, first 3):")
pp.pprint(facts_from_csv[:3])


FACTS (csv, first 3):
[FactInstance(predicate_id='eaeeb51498c9e64567c571615e686d046eef8804c6f1b2a86d2ee2f4358f0829',
              terms=[Const(value='alice', datatype='string'),
                     Const(value='openai', datatype='string')],
              prob=1.0),
 FactInstance(predicate_id='eaeeb51498c9e64567c571615e686d046eef8804c6f1b2a86d2ee2f4358f0829',
              terms=[Const(value='bob', datatype='string'),
                     Const(value='acme_corp', datatype='string')],
              prob=1.0),
 FactInstance(predicate_id='eaeeb51498c9e64567c571615e686d046eef8804c6f1b2a86d2ee2f4358f0829',
              terms=[Const(value='carol', datatype='string'),
                     Const(value='globex', datatype='string')],
              prob=1.0)]


### Define Rule Instances

##### 3.1 Define rules directly

In [ ]:
# === Construct rules (Head var-only) ===
# Usage:
# - HeadSchema defines the head predicate + variables (var-only).
# - Body is a list of literals with clause-level probability.
#
# Design rationale:
# - "head var-only" prevents constants in the head; constants must appear in bodies.
# - Multiple bodies represent multiple rule branches (Head :- Body_i).
import pprint as pp

from new_symbolic_agent.ir.fact_schema import ArgSpec, PredicateSchema
from new_symbolic_agent.ir.expr_ir import Var, Const, Call, Unify, If, NotExpr
from new_symbolic_agent.ir.rule_schema import RefLiteral, ExprLiteral, HeadSchema, Body, Rule

head_pred = PredicateSchema("relocation_candidate", 2, [ArgSpec("string"), ArgSpec("string")])
head = HeadSchema(predicate=head_pred, terms=[Var("X"), Var("Y")])

# Body1: WorksAt(X, C) and CompanyHQ(C, Y)
# - Reuse head vars in body to ensure logical linkage.
body1 = Body(
    literals=[
        RefLiteral(predicate_id=works.schema_id, terms=[Var("X", "string"), Var("C", "string")]),
        RefLiteral(predicate_id=hq.schema_id, terms=[Var("C", "string"), Var("Y", "string")]),
    ],
    prob=0.7,
)

# Body2: Social + location signals and an if-then-else expression
# - ExprLiteral holds structured ExprIR (no raw strings).
expr = If(
    cond=Call("eq", [Var("Y", "string"), Const("seattle", "string")]),
    then=Unify(Var("Flag", "bool"), Const(True, "bool")),
    else_=Unify(Var("Flag", "bool"), Const(False, "bool")),
)
body2 = Body(
    literals=[
        RefLiteral(predicate_id=lives.schema_id, terms=[Var("X", "string"), Var("Y", "string")]),
        RefLiteral(predicate_id=friend.schema_id, terms=[Var("X", "string"), Var("F", "string")]),
        RefLiteral(predicate_id=lives.schema_id, terms=[Var("F", "string"), Var("Y", "string")]),
        RefLiteral(predicate_id=located.schema_id, terms=[Var("Y", "string"), Var("Country", "string")]),
        ExprLiteral(expr=expr),
    ],
    prob=None,  # will use default rule prob during render
)

rule_from_user = Rule(head=head, bodies=[body1, body2])

print("\nRULE (user):")
pp.pprint(rule_from_user)


RULE (user):
Rule(head=HeadSchema(predicate=PredicateSchema(name='relocation_candidate',
                                               arity=2,
                                               signature=[ArgSpec(datatype='string',
                                                                  role=None,
                                                                  namespace=None),
                                                          ArgSpec(datatype='string',
                                                                  role=None,
                                                                  namespace=None)],
                                               description=None),
                     terms=[Var(name='X', datatype=None),
                            Var(name='Y', datatype=None)]),
     bodies=[Body(literals=[RefLiteral(predicate_id='eaeeb51498c9e64567c571615e686d046eef8804c6f1b2a86d2ee2f4358f0829',
                                       terms=[Var(name='X',

##### 3.2 Literal syntax

In [7]:
# === Literal syntax examples (all forms) ===
# Usage:
# - Demonstrate all literal/expr forms: RefLiteral, negated RefLiteral,
#   Unify, Call, If, NotExpr.
#
# Design rationale:
# - Keep this section lightweight by showing literal instances only.
# - These literals can be combined into bodies to form full rules.
print("\nLITERAL SYNTAX EXAMPLES (all forms):")

literals = [
    # RefLiteral (positive)
    RefLiteral(predicate_id=lives.schema_id, terms=[Var("X"), Var("Y")]),
    # RefLiteral (negated)
    RefLiteral(predicate_id=lives.schema_id, terms=[Var("X"), Var("Y")], negated=True),
    # ExprLiteral: Unify (equality/assignment)
    ExprLiteral(expr=Unify(Var("Y"), Const("seattle", "string"))),
    # ExprLiteral: Call (comparison)
    ExprLiteral(expr=Call("eq", [Var("Y"), Const("seattle", "string")])),
    # ExprLiteral: Call (arithmetic) + Unify
    ExprLiteral(expr=Unify(Var("Y", "int"), Call("add", [Var("X", "int"), Const(1, "int")]))),
    # ExprLiteral: If (if-then-else)
    ExprLiteral(
        expr=If(
            cond=Call("gt", [Var("X", "int"), Const(0, "int")]),
            then=Unify(Var("Y", "int"), Call("add", [Var("X", "int"), Const(1, "int")])),
            else_=Unify(Var("Y", "int"), Const(0, "int")),
        )
    ),
    # ExprLiteral: NotExpr (expression-level negation)
    ExprLiteral(expr=NotExpr(Call("eq", [Var("Y"), Const("seattle", "string")]))),
]

for idx, literal in enumerate(literals, start=1):
    print(f"\nLiteral Example {idx}:")
    pp.pprint(literal)


LITERAL SYNTAX EXAMPLES (all forms):

Literal Example 1:
RefLiteral(predicate_id='068d3061762f15df68451a09562599525ac2b4f007daf64dd71d2cd0062a1036',
           terms=[Var(name='X', datatype=None), Var(name='Y', datatype=None)],
           negated=False)

Literal Example 2:
RefLiteral(predicate_id='068d3061762f15df68451a09562599525ac2b4f007daf64dd71d2cd0062a1036',
           terms=[Var(name='X', datatype=None), Var(name='Y', datatype=None)],
           negated=True)

Literal Example 3:
ExprLiteral(expr=Unify(lhs=Var(name='Y', datatype=None),
                       rhs=Const(value='seattle', datatype='string')))

Literal Example 4:
ExprLiteral(expr=Call(op='eq',
                      args=[Var(name='Y', datatype=None),
                            Const(value='seattle', datatype='string')]))

Literal Example 5:
ExprLiteral(expr=Unify(lhs=Var(name='Y', datatype='int'),
                       rhs=Call(op='add',
                                args=[Var(name='X', datatype='int'),
          

##### Libraries

In [ ]:
from new_symbolic_agent.rules.library import Library, LibrarySpec
from new_symbolic_agent.rules.library_runtime import LibraryRuntime

# === Libraries and runtime libraries ===
# Usage:
# - Library holds special predicates that can be referenced in rules but are not part of the FactSchema.
# - Runtime library provides implementations for these predicates during rendering.
# Design rationale:
# - This allows us to reference external predicates (e.g., "member") without bloating the FactSchema.

lib = Library()
lib.register(
    LibrarySpec(
        name="member",
        arity=2,
        kind="predicate",
        description="Check if an element is a member of a list",
        signature=["term", "list"],
    )
)
lib.register(
    LibrarySpec(
        name="is_even",
        arity=1,
        kind="expr",
        description="Check if a number is even",
        signature=["term"],
    )
)

runtime = LibraryRuntime(lib)
runtime.register(
    name="member",
    arity=2,
    kind="predicate",
    backend="problog",
    handler=lambda args: f"member({args[0]}, {args[1]})",
)

### Langda Generation

##### Schemas for Constrained Decoding

In [ ]:
# === Constraint schemas (LLM decoding) ===
# Usage:
# - Library holds predicate-like or expr-like specs (metadata).
# - Constraint schemas are generated to restrict LLM output.
#
# Design rationale:
# - LibrarySpec is serializable metadata for prompt + schema use.
# - Runtime implementations are attached later in LibraryRuntime for rendering.
import pprint as pp

from new_symbolic_agent.rules.constraint_schemas import (
    build_pydantic_rule_model,
    build_responses_schema,
    build_predicate_catalog,
)

json_schema = build_responses_schema(view, lib, mode="compact")
pydantic_model = build_pydantic_rule_model(view, lib, mode="compact")
catalog = build_predicate_catalog(view, lib)  # prompt-only catalog

print("\n### JSON SCHEMA (responses):")
pp.pprint(json_schema)
print("\n")
print("\n### PYDANTIC MODEL:")
pp.pprint(pydantic_model.schema())
print("\n")
print("\n### PREDICATE CATALOG (prompt-only):")
pp.pprint(catalog)



### JSON SCHEMA (responses):
{'$defs': {'call': {'additionalProperties': False,
                    'properties': {'args': {'items': {'$ref': '#/$defs/expr'},
                                            'type': 'array'},
                                   'kind': {'const': 'call', 'type': 'string'},
                                   'op': {'type': 'string'}},
                    'required': ['args', 'kind', 'op'],
                    'type': 'object'},
           'const': {'additionalProperties': False,
                     'properties': {'datatype': {'type': 'string'},
                                    'kind': {'const': 'const',
                                             'type': 'string'},
                                    'value': {'type': ['string',
                                                       'number',
                                                       'boolean']}},
                     'required': ['datatype', 'kind', 'value'],
                     'type': 'o

/var/folders/05/6btr2vg13b9gvgs3gxt8fw_40000gn/T/ipykernel_53981/3551137814.py:25: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  pp.pprint(pydantic_model.schema())


##### USER INPUT INSTRUCTION

In [10]:
head_pred_for_agent = PredicateSchema("relocation_candidate", 2, [ArgSpec("string"), ArgSpec("string")])
head_for_agent = HeadSchema(predicate=head_pred_for_agent, terms=[Var("X"), Var("Y")])

instructions = (
    "Decide whether a person should be considered a relocation candidate for a city. "
    "Use real-world signals such as where they work, where the company's headquarters are, "
    "where they currently live, and whether they have close connections in the target city. "
    "Consider that cities in the same country may be more plausible. "
    "Provide multiple plausible cases with probabilities."
)

##### Prompts

In [11]:
# === LLM prompt example (head is fixed externally) ===
# Usage:
# - The LLM only outputs bodies. The head is fixed outside decoding.
# - Use catalog for human/LLM prompt, schema for strict decoding.
#
# Design rationale:
# - Splitting head/bodies avoids complex dependency analysis in generation.
# - Compact mode keeps schema small and stable.
import json

from openai import OpenAI

head_vars = ", ".join([t.name for t in head_for_agent.terms])
head_params = ", ".join([a.datatype for a in head_for_agent.predicate.signature])

system_prompt = """
You output ONLY JSON that conforms to the provided JSON Schema (no extra text).
Context:
- The predicate head is fixed externally and must NOT appear in the output.
- You are given a fact-layer schema and nodes/relationships as formatted data.
- You are also given a predicate registry. You MUST reference only existing predicates from the registry.

Task:
- Generate one or more cases (bodies) as conditional statements to solve the user task.
- Each case has: probability (0..1) and conditions (structured).
- Reuse the head variable names when possible (to link head and body).
"""

user_prompt = f"""
## Fixed head (do NOT output it):
- Head Name: {head_for_agent.predicate.name}
- Head Arity: {head_for_agent.predicate.arity}
- Head Param Types: {head_params}
- Head Vars: {head_vars}

## Known predicates:
{json.dumps(catalog, ensure_ascii=False, indent=2)}

## Instructions:
{instructions}
"""

print("\nSYSTEM PROMPT:")
print(system_prompt.strip())
print("\nUSER PROMPT:")
print(user_prompt.strip())



SYSTEM PROMPT:
You output ONLY JSON that conforms to the provided JSON Schema (no extra text).
Context:
- The predicate head is fixed externally and must NOT appear in the output.
- You are given a fact-layer schema and nodes/relationships as formatted data.
- You are also given a predicate registry. You MUST reference only existing predicates from the registry.

Task:
- Generate one or more cases (bodies) as conditional statements to solve the user task.
- Each case has: probability (0..1) and conditions (structured).
- Reuse the head variable names when possible (to link head and body).

USER PROMPT:
## Fixed head (do NOT output it):
- Head Name: relocation_candidate
- Head Arity: 2
- Head Param Types: string, string
- Head Vars: X, Y

## Known predicates:
{
  "eaeeb51498c9e64567c571615e686d046eef8804c6f1b2a86d2ee2f4358f0829": {
    "name": "works_at",
    "arity": 2,
    "arg_types": [
      "string",
      "string"
    ],
    "description": "Relationship: person works at company"


##### Call Openai Api

In [12]:
client = OpenAI()
resp = client.responses.create(
    model="gpt-4o-2024-08-06",
    instructions=system_prompt,
    input=user_prompt,
    text={
        "format": {
            "type": "json_schema",
            "name": "cases_only_contract",
            "schema": json_schema,
            "strict": True,
        }
    },
)

##### Convert output of llm

In [ ]:
# === Convert LLM output -> Rule IR ===
# Usage:
# - Use resp_to_rule to parse Responses API output into Rule IR.
#
# Design rationale:
# - Parsing goes through Pydantic validation to guarantee structure.

import pprint as pp

from new_symbolic_agent.examples.parse_llm_response import resp_to_rule

# rule_from_agent = resp_to_rule(
#     _MockResp(), head=head_for_agent, view=view, library=lib, mode="compact"
# )
rule_from_agent = resp_to_rule(
    resp, head=head_for_agent, view=view, library=lib, mode="compact"
)
print("\nRULE (from LLM):")
pp.pprint(rule_from_agent)



RULE (from LLM):
Rule(head=HeadSchema(predicate=PredicateSchema(name='relocation_candidate',
                                               arity=2,
                                               signature=[ArgSpec(datatype='string',
                                                                  role=None,
                                                                  namespace=None),
                                                          ArgSpec(datatype='string',
                                                                  role=None,
                                                                  namespace=None)],
                                               description=None),
                     terms=[Var(name='X', datatype=None),
                            Var(name='Y', datatype=None)]),
     bodies=[Body(literals=[RefLiteral(predicate_id='068d3061762f15df68451a09562599525ac2b4f007daf64dd71d2cd0062a1036',
                                       terms=[Var(name=

##### Validate

In [ ]:
# === Validate rules ===
# Usage:
# - RuleValidator enforces: FactView references, arity, recursion rules, etc.
#
# Design rationale:
# - LLM output is always validated before execution or rendering.
from new_symbolic_agent.rules.validator import RuleValidator

RuleValidator(view, library=lib).validate(rule_from_agent)


### Render as logic code

##### Render as problog code

In [ ]:
# === Render ===
# Usage:
# - Renderers turn Rule IR into backend-specific code.
# - LibraryRuntime supplies backend handlers for library predicates/exprs.
#
# Design rationale:
# - Rendering is decoupled from IR so new backends can be plugged in.
from new_symbolic_agent.probability import ProbabilityConfig
from new_symbolic_agent.rules.library_runtime import LibraryRuntime
from new_symbolic_agent.mappers.renderers import (
    ProbLogRenderer,
    PrologRenderer,
    DatalogRenderer,
    CypherRenderer,
    RenderContext,
)
from new_symbolic_agent.ir.rule_schema import Query

render_cfg = ProbabilityConfig(default_rule_prob=0.6, missing_prob_policy="inject_default")
ctx = RenderContext(schema=schema, library=lib, library_runtime=runtime, prob_config=render_cfg)

all_facts = facts_from_user + facts_from_csv
all_rules = [rule_from_user, rule_from_agent]
all_queries = [
    Query(predicate=head_pred, terms=[Var("X"), Var("Y")]),
    Query(predicate_id=person.schema_id, terms=[Const("alice", "string")]),
]

problog_text = ProbLogRenderer().render_program(all_facts, all_rules, ctx, queries=all_queries)
print("\nPROBLOG (facts + rules):")
print(problog_text)



PROBLOG (facts + rules):
0.9::person(alice).
1.0::person(bob).
1.0::city(seattle).
1.0::company(openai).
1.0::country(usa).
1.0::works_at(alice, openai).
1.0::works_at(bob, acme_corp).
1.0::works_at(carol, globex).
1.0::works_at(dave, initech).
1.0::works_at(emma, umbrella).
1.0::works_at(frank, stark_industries).
1.0::works_at(grace, wayne_enterprises).
1.0::works_at(heidi, wonka).
1.0::works_at(ivan, tyrell).
1.0::works_at(judy, cyberdyne).
1.0::works_at(mallory, openai).
1.0::works_at(niaj, acme_corp).
1.0::works_at(olivia, globex).
1.0::works_at(peggy, initech).
1.0::works_at(quinn, umbrella).
1.0::works_at(ruth, stark_industries).
1.0::works_at(sybil, wayne_enterprises).
1.0::works_at(trent, wonka).
1.0::works_at(victor, tyrell).
1.0::works_at(walter, cyberdyne).
1.0::works_at(xavier, openai).
1.0::works_at(yvonne, acme_corp).
1.0::works_at(zara, globex).
1.0::works_at(oscar, initech).
1.0::located_in(seattle, usa).
1.0::located_in(san_francisco, usa).
1.0::located_in(new_york, u

##### execute problog code

In [16]:
from problog.program import PrologString
from problog import get_evaluatable

result = get_evaluatable().create_from(PrologString(problog_text)).evaluate()
print("\nPROBLOG EVALUATION RESULT:")
pp.pprint(result)


PROBLOG EVALUATION RESULT:
{relocation_candidate(alice,san_francisco): 0.7,
 relocation_candidate(bob,chicago): 0.7,
 relocation_candidate(carol,new_york): 0.7,
 relocation_candidate(dave,austin): 0.7,
 relocation_candidate(emma,boston): 0.7,
 relocation_candidate(frank,los_angeles): 0.7,
 relocation_candidate(grace,dallas): 0.7,
 relocation_candidate(heidi,denver): 0.7,
 relocation_candidate(ivan,seattle): 0.7,
 relocation_candidate(judy,houston): 0.7,
 relocation_candidate(mallory,san_francisco): 0.7,
 relocation_candidate(niaj,chicago): 0.7,
 relocation_candidate(olivia,new_york): 0.7,
 relocation_candidate(peggy,austin): 0.7,
 relocation_candidate(quinn,boston): 0.7,
 relocation_candidate(ruth,los_angeles): 0.7,
 relocation_candidate(sybil,dallas): 0.7,
 relocation_candidate(trent,denver): 0.7,
 relocation_candidate(victor,seattle): 0.7,
 relocation_candidate(walter,houston): 0.7,
 relocation_candidate(xavier,san_francisco): 0.7,
 relocation_candidate(yvonne,chicago): 0.7,
 reloca

##### Render as other codes...

In [ ]:
# Other backends (stubs for now) not implemented yet
from new_symbolic_agent.mappers.renderers import RenderError
try:
    _ = PrologRenderer().render_rule(rule_from_agent, ctx)
    _ = DatalogRenderer().render_rule(rule_from_agent, ctx)
    _ = CypherRenderer().render_rule(rule_from_agent, ctx)
except RenderError as e:
    print("\nOther renderers not implemented yet.")


Other renderers not implemented yet.
